In [1]:
## Ex 5-1. QPushButton.

import sys
from PyQt5.QtGui import *
from PyQt5.QtCore import *
from PyQt5.QtWidgets import *
from PyQt5 import uic
import numpy as np
import qrng

qrng.set_provider_as_IBMQ('c727c942f774f0acc34e5103c6c1591cd129619e3827a9d3a7d29b47d42008519704d67e276558c30b4c4ee1ce1d3f890267f10f3af17c8735c8ca877118deda')
qrng.set_backend('ibm-q')

CalUI = 'C:/Users/cbseo/Desktop/Indian poQer/module/button.ui'

class MainDialog(QDialog):
    def __init__(self):
        QDialog.__init__(self, None, Qt.WindowStaysOnTopHint)
        uic.loadUi(CalUI, self)
        
        self.Init()
        self.Choiseline.setText("Press the Start button")
        self.Button_start.clicked.connect(lambda state, Button = self.Button_start :self.Clicked_Start(state, Button))
        self.Button_deal.clicked.connect(lambda state, Button = self.Button_deal :self.Clicked_Deal(state, Button))
        self.Button_retry.clicked.connect(lambda state, Button = self.Button_retry :self.Clicked_Retry(state, Button))
        self.Button_upbet.clicked.connect(lambda state, Button = self.Button_upbet :self.Clicked_Bet(state, Button))
        self.Button_downbet.clicked.connect(lambda state, Button = self.Button_downbet :self.Clicked_Bet(state, Button))     
        self.Button_raise.clicked.connect(lambda state, Button = self.Button_raise :self.Clicked_Command(state, Button))
        self.Button_call.clicked.connect(lambda state, Button = self.Button_call :self.Clicked_Command(state, Button))
        self.Button_fold.clicked.connect(lambda state, Button = self.Button_fold :self.Clicked_Command(state, Button))
   
    def Init(self):
        self.Turn = 0
        self.Bet = 0
        self.Plchip = 20
        self.Aichip = 20
        self.card = 0
        self.Plcardlist = np.zeros(10, int)
        self.Aicardlist = np.zeros(10, int)
        self.Pot = 0
        self.Signal = 0
        self.Plselect = 0
        self.Aiselect = 0
        self.Aicardurl = ""
        self.Plcardurl = ""
        self.Maxbet = 0
    
    def Button_Clicked(self, state, Button):
        self.Turnline.clear()
        self.Betline.clear()
        self.Aicard.clear()
        self.Plcard.clear()
        self.Aichipline.clear()
        self.Plchipline.clear()
        self.Potchipline.clear()
        self.Choiseline.clear()
        
    def Line_State(self, state, Button):
        self.Turnline.setText("Turn "+ str(self.Turn) + "   -   your turn")
        self.Betline.setText(str(self.Bet) + " bet")
        self.Aichipline.setText(str(self.Aichip) + "chip")
        self.Plchipline.setText(str(self.Plchip) + "chip")
        self.Potchipline.setText(str(self.Pot) + "chip")
        
    def Clicked_Command(self, state, Button):
        if (self.Signal == 2):
            self.Button_Clicked(state, Button)
            if (Button.text() == "RAISE"):
                self.Aichip -= self.Bet
                self.Plchip -= self.Bet
                self.Pot += self.Bet * 2
                self.Bet = 0
                self.Betline.setText("0 bet")
            elif (Button.text() == "CALL"):
                self.Plcard.setStyleSheet('image:url({}); border:0px;'.format(str(self.Plcardurl)))
                if(self.Aiselect == self.Plselect):
                    self.Plchip += int(self.Pot/2)
                    self.Aichip += int(self.Pot/2)
                    self.Turnline.clear()
                    self.Turnline.setText("DRAW!!")
                elif(self.Aiselect > self.Plselect):
                    self.Aichip += self.Pot
                    self.Turnline.clear()
                    self.Choiseline.clear()
                    self.Turnline.setText("You Lose!!")
                    self.Choiseline.setText("You earn " + str(self.Pot) + " coin :(")
                elif(self.Aiselect < self.Plselect):
                    self.Plchip += self.Pot
                    self.Turnline.clear()
                    self.Choiseline.clear()
                    self.Turnline.setText("You Win!!")
                    self.Choiseline.setText("You earn " + str(self.Pot) + " coin :)")
                self.Aiselect = 0
                self.Plselect = 0
                self.Bet = 0
                self.Pot = 0
                self.Maxbet = 0
                self.Betline.setText(str(self.Bet) + " bet")
                self.Aichipline.setText(str(self.Aichip) + "chip")
                self.Plchipline.setText(str(self.Plchip) + "chip")
                self.Potchipline.setText(str(self.Pot) + "chip")
            elif (Button.text() == "FOLD"):
                self.Aichip += self.Pot
            self.Line_State(state, Button)
            self.Choiseline.setText("You " + Button.text())
        
    def Clicked_Bet(self, state, Button):
        if(self.Signal == 2):
            self.Button_Clicked(state, Button)
            if (Button.text() == "UP"):
                if(self.Maxbet == 5):
                    self.Choiseline.clear()
                    self.Choiseline.setText("You have to press Raise or CALL because the maximum betting cannot exceed 5chip.")
                elif(self.Bet+1 < self.Plchip and self.Bet+1 < self.Aichip and self.Bet < 5):
                    self.Bet += 1
                    self.Maxbet += 1
            elif (Button.text() == "DOWN"):
                if(self.Bet > 0):
                    self.Bet -= 1
            self.Line_State(state, Button)
            self.Betline.setText(str(self.Bet) + " bet")
    
    def Clicked_Start(self, state, Button):
        self.Button_Clicked(state, Button)
        self.Turnline.setText("Welcome")
        self.Betline.setText(str(self.Bet) + " bet")
        self.Aichipline.setText(str(self.Aichip) + " chip")
        self.Plchipline.setText(str(self.Plchip) + " chip")
        self.Potchipline.setText(str(self.Pot) + " chip")
        self.Choiseline.setText("Press the Deal button to start the game")
        self.Signal = 1
        
    def Clicked_Deal(self, state, Button):
        if(self.Signal >= 1 and self.Aichip != 0):
            self.Turn += 1
            self.Aichip -= 1
            self.Plchip -= 1
            self.Pot += 2
            self.Button_Clicked(state, Button)
            while (self.Plselect == 0 and self.Aiselect == 0):
                self.Plselect = self.Card_Select(state, Button, self.Plcardlist)
                self.Aiselect = self.Card_Select(state, Button, self.Aicardlist)
                if(self.Plselect == 0 and self.Aiselect == 0):
                    for i in range(10):
                        self.Plcardlist[i] = 0
                        self.Aicardlist[i] = 0
            self.Aicardurl = "C:/Users/cbseo/Desktop/Indian PoQer/card_image/H" + str(self.Aiselect) +".png"
            self.Plcardurl = "C:/Users/cbseo/Desktop/Indian PoQer/card_image/S" + str(self.Plselect) +".png"
            self.Aicard.setStyleSheet('image:url({}); border:0px;'.format(str(self.Aicardurl)))
            self.Plcard.setStyleSheet('image:url(C:/Users/cbseo/Desktop/Indian PoQer/card_image/back.png); border:0px;')
            self.Line_State(state, Button)
            self.Signal = 2
        elif(self.Aichip == 0):
            self.Turnline.clear()
            self.Choiseline.clear()
            self.Turnline.setText("Congratulations! You won!")
            self.Choiseline.setText("If you want to play more games, press the Retry button to start a new game.")
    
    def Clicked_Retry(self, state, Button):
        if(self.Signal == 1 or self.Signal == 2):
            self.Init()
            self.Button_Clicked(state, Button)
            self.Clicked_Start(state, Button)
        
    def Card_Select(self, state, Button, Player):
        while (1):
            self.card = qrng.get_random_int(1,10)
            if(Player[self.card-1] == 0):
                Player[self.card-1] = 1
                return self.card
            else :
                continue
                return 0
            break
        
app = QApplication(sys.argv)
main_dialog = MainDialog()
main_dialog.show()
app.exec_()

configrc.store_credentials:WARNING:2021-02-19 14:52:14,954: Credentials already present. Set overwrite=True to overwrite.
C:\Users\cbseo\anaconda3\envs\qiskit\lib\site-packages\qiskit\providers\ibmq\ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


0